In [1]:
import matplotlib as plt 
import numpy as np 
from matplotlib import colors
from PIL import Image, ImageDraw
import random
import pandas as pd
from collections import defaultdict
import random
import copy

In [2]:
def createList(r1, r2):
        if (r1 == r2):
            return r1
        else:
            res = []
            while(r1 != r2):
                res.append(r1)
                r1 += 1
                if (r1 > 50):
                    r1 = r1-50
            res.append(r2)
        return res 

In [3]:
class Graph():
    
    def __init__(self):
        
        #create graph
        self.graph = {}
        for i in range(1,51):
            if i-1 == 0:
                self.graph[i] = [2, 50]
            elif i+1 == 51:
                self.graph[i] = [1, 49]
            else:
                self.graph[i] = [i+1, i-1]
        
        #create list of nodes
        self.nodes = createList(1, 50)
                
    def addEdge(self):
        boo1 = True
        while boo1:
            if len(self.nodes) == 0:
                boo1 = False
                break
            val = random.choice(self.nodes)
            self.nodes.remove(val)
            valstart = val-5
            valend = val+5
            if (valstart < 1):
                valstart = valstart+50
            if (valend > 50):
                valend = valend-50
            val_nodes = createList(valstart, valend)
            if(val == 1):
                val_nodes.remove(50)
            elif(val == 50):
                val_nodes.remove(1)
            else:
                val_nodes.remove(val-1)
                val_nodes.remove(val+1)
            boo2 = False
            while len(val_nodes) > 0:
                newnode = random.choice(val_nodes)
                if newnode in self.nodes:
                    self.nodes.remove(newnode)
                    self.graph[val].append(newnode)
                    self.graph[newnode].append(val)
                    boo2 = True
                    break
                else:
                    val_nodes.remove(newnode)
                    
            if boo2:
                break
        if boo1 == False:
            return False
        else:
            return True

In [4]:
#start position, end position, backwards path to take
#returns path
def BFS(start, end, graph):
 
    # a queue of the vertices whose path is to be scanned
    # boolean array for checking if node has been visited at least once or not
    # all moves in path set to -1
    queue = []
    visited = [False for i in range(51)]
    back_path = [-1 for i in range(51)]
    
    # start is visited and added to the queue
    visited[start] = True
    queue.append(start)

    # BFS algorithm - look at all neighbors first
    while (len(queue) != 0):
        u = queue[0]
        queue.pop(0)
        for i in range(len(graph[u])):
            if (visited[graph[u][i]] == False):
                visited[graph[u][i]] = True
                back_path[graph[u][i]] = u
                queue.append(graph[u][i])

                # We stop BFS when we find
                # destination.
                if (graph[u][i] == end):
                    path = []
                    crawl = end
                    path.append(crawl)
                    
                    #reversing back_path and removing -1 from nodes that are not in the path to get full path
                    while (back_path[crawl] != -1):
                        path.insert(0, back_path[crawl])
                        crawl = back_path[crawl]
                    return path

    return [end]

In [5]:
class Predator():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure that predator doesn't start where the agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def pred_move(self, g, agentpos):
        
        #getting all next possible moves of predator
        neighbors = g.graph[self.pos]
        path = []
        shortest_path = []
        path_length = 50
        #finding shortest path to agent using BFS
        for neighbor in neighbors:
            path = BFS(neighbor, agentpos, g.graph)
            
            #saving shortest path(s)
            if(len(path) <= path_length):
                if(len(path) == path_length):
                    shortest_path.append(path)
                else:
                    shortest_path = []
                    shortest_path.append(path)
                    path_length = len(path)
        
        #randomly picking path of the shortest paths and moving to next neighbor
        new_path = random.randint(0,(len(shortest_path)-1))
        self.pos = shortest_path[new_path][0]
        return self

    def pred_distracted_move(self, g, agentpos): #predator moving with probability 0.6 to node closer to agent and probability 0.4 to any other random neighbour
        
        #getting all next possible moves of predator
        neighbors = g.graph[self.pos]
        path = []
        shortest_path = []
        path_length = 50
        #finding shortest path to agent using BFS
        for neighbor in neighbors:
            path = BFS(neighbor, agentpos, g.graph)
            
            #saving shortest path(s)
            if(len(path) <= path_length):
                if(len(path) == path_length):
                    shortest_path.append(path)
                else:
                    shortest_path = []
                    shortest_path.append(path)
                    path_length = len(path)
        
        #randomly picking path of the shortest paths 
        new_path = random.randint(0,(len(shortest_path)-1))
        chance = random.randint(1,100)
        if chance <= 60: #60 percent chance of moving correctly
            self.pos = shortest_path[new_path][0]
        else: #40 percent chance of moving in distracted manner
            self.pos = random.choice(neighbors)
        return self

In [6]:
class Prey():
    
    def __init__(self, agentloc):
        self.pos = random.randint(1,50)
        #making sure prey doesn't start where agent is
        while self.pos == agentloc:
            self.pos = random.randint(1,50)
        
    def prey_move(self, g):
        prey_loc_neighbours = [self.pos]
        for i in g.graph[self.pos]:
            prey_loc_neighbours.append(i)
        #randomly choose neighbor to move to (or staying put)
        self.pos = random.choice(prey_loc_neighbours)
        return self

In [89]:
def Agent5(g):
    agent_pos = random.randint(1,50)
    predator_5 = Predator(agent_pos)
    prey_5 = Prey(agent_pos)
    fail = 0
    steps = 0
    path = []
    path.append(agent_pos)
    fail_list = []

    belief_vector =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
        if i == predator_5.pos-1:
            belief_vector[i] = 1
        else:
            belief_vector[i] = 0
    
    #print(sum(belief_vector))
    
    while(fail == 0 and steps <= 100):
        max_prob = max(belief_vector)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob = []
        for itr in range(len(belief_vector)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector[itr] == max_prob):
                list_of_positions_max_prob.append(itr+1)

        #chosing node with highest prob of having pred closest to agent position
        pat_len = 50
        best_positions = []
        for position in list_of_positions_max_prob:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_positions.append(position)
                else:
                    best_positions = []
                    best_positions.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))
        
        possible_pred_pos = random.choice(best_positions) #randomly selecting one of those closest positions and surveying it
        #print(possible_pred_pos)
        
        #surveying then updating belief
        if possible_pred_pos == predator_5.pos: #if we survey node and find predator
            for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
                if i == predator_5.pos-1:
                    belief_vector[i] = 1
                else:
                    belief_vector[i] = 0
        else:
                pred_not = 1-max_prob #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
                belief_vector[possible_pred_pos-1] = 0
                for i in range(len(belief_vector)):
                        belief_vector[i] = belief_vector[i]/pred_not

        #print(str(belief_vector) + ' first update during timestep')
        #print(str(sum(belief_vector)) + " Update 1 Timestep: " + str(steps))
        #if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
        #    print(str(sum(belief_vector)) + " Update 1 Timestep: " + str(steps))
        #    fail_list.append((1,steps))


        """quasi_1 = sum(belief_vector)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_1""" #to ensure that belief vector sums up to 1, leaving no margin for error

        list_of_positions_max_chance = []
        for itr in range(len(belief_vector)): #making a list of positions which now have the highest probability of having the predator
            if(belief_vector[itr] == max(belief_vector)):
                list_of_positions_max_chance.append(itr+1)

        #chosing node with highest prob of having pred closest to us
        pat_len = 50
        best_chance = []
        for position in list_of_positions_max_chance:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_chance.append(position)
                else:
                    best_chance = []
                    best_chance.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        predicted_pred_pos = random.choice(best_chance) #randomly selecting one of those closest neighbour positions and assuming it as predator position
        
        #Agent 1 from here on
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        moves_1 = []
        moves_2 = []
        moves_3 = []
        moves_4 = []
        moves_5 = []
        moves_6 = []
        path_to_prey = []
        path_to_predator = []
        curr_to_pred = len(BFS(agent_pos, predicted_pred_pos, g.graph))
        curr_to_prey = len(BFS(agent_pos, prey_5.pos, g.graph))

            
        for neighbor in neighbors:
            
            # shortest path from neighbor to prey and shortest path from neighbor to predator
            path_to_prey = BFS(neighbor, prey_5.pos, g.graph)
            path_to_predator = BFS(neighbor, predicted_pred_pos, g.graph)
            
            #print("Prey and Predator Moves")
            #print(path_to_prey)
            #print(path_to_predator)
            if (len(path_to_prey) < curr_to_prey and len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_1.append(tup)
            
            elif (len(path_to_prey) < curr_to_prey and len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_2.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_3.append(tup)

            elif (len(path_to_prey) == curr_to_prey and len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_4.append(tup)

            elif (len(path_to_predator) > curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_5.append(tup)

            elif (len(path_to_predator) == curr_to_pred):
                tup = (len(path_to_prey), len(path_to_predator), neighbor)
                moves_6.append(tup)          

        
        #print()
        if(len(moves_1) > 0):
            moves_sorted = sorted(moves_1, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_2) > 0):
            moves_sorted = sorted(moves_2, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_3) > 0):
            moves_sorted = sorted(moves_3, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_4) > 0):
            moves_sorted = sorted(moves_4, key=lambda t: (t[0], -t[1]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_5) > 0):
            moves_sorted = sorted(moves_5, key=lambda t: (-t[1],t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]

        elif(len(moves_6) > 0):
            moves_sorted = sorted(moves_6, key=lambda t: (-t[1], t[0]))
            #print(moves_sorted)
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]
        
        path.append(agent_pos)

        steps += 1

        if (agent_pos == predator_5.pos):
            fail = 1 #if agent has already moved to location of pred
            break
        elif (agent_pos == prey_5.pos):
            break
        else: #treat as survey and update belief
            pred_not = 1-belief_vector[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector[agent_pos-1] = 0
            for i in range(len(belief_vector)):
                    belief_vector[i] = belief_vector[i]/pred_not
            
        #print(str(sum(belief_vector)) + " Update 2 Timestep: " + str(steps))
        #if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
        #    print(str(sum(belief_vector)) + " Update 2 Timestep: " + str(steps))
        #    fail_list.append((2,steps))        
        #print(str(belief_vector) + ' first update during timestep')
        #print(sum(belief_vector))

        """quasi_3 = sum(belief_vector)
        #print(quasi)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_3""" #to ensure that belief vector sums up to 1, leaving no margin for error
            

        #move predator and prey
        prey_5.prey_move(g)

        if (prey_5.pos == agent_pos): #if prey luckily moves to location of agent
            break

        predator_5.pred_distracted_move(g, agent_pos)

        if(predator_5.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        #update belief for next timestep
        temp_belief_vector = belief_vector.copy() #keeping copy for correct calculations
        for i in range(len(belief_vector)):
            mag_sum=0 #predator wont be at same node during next timestep
            i_neighbours = g.graph[i+1] 
            for bloke in i_neighbours:
                bloke_neighbours = g.graph[bloke] #finding neighbours of neighbours
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours:
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph) #performing bfs from these 2nd degree neighbours
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n) #keep list of neighbours with shortest path
                            shortest_distance = len(worthy_bloke_path) 
                mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(bloke_neighbours))) #moving from node bloke to node i in random manner
                mag_sum = mag_sum + (0.6*(temp_belief_vector[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes)) #moving from node bloke to node i because it is on the shortest path to Agent
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector[i] = mag_sum #assigning probability
        #print(str(belief_vector) + ' last update before next timestep')
        #print(str(sum(belief_vector)) + " Update 3 Timestep: " + str(steps))

        #if (sum(belief_vector) > 1.01 or sum(belief_vector) < 0.99):
            #print(str(sum(belief_vector)) + " Update 3 Timestep: " + str(steps))
            #fail_list.append((3,steps+1))        

        quasi_2 = sum(belief_vector)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_2
        #print(belief_vector)
    
    #if (len(fail_list) > 0):
        #print("Fail List: " + str(fail_list))#
        
    if(steps > 100):
        fail = 2     
    return fail, path

In [90]:
wins = 0
losses = 0
timeout = 0
times = 0
win_dist = []
loss_dist = []

while times < 3000:
    g = Graph()
    graph = g.addEdge()
    while graph:
        graph = g.addEdge()
    res, path = Agent5(g)
    
    if(res == 0):
        wins += 1
        win_dist.append(len(path))
    elif(res == 1):
        losses += 1
        loss_dist.append(len(path))
    else:
        timeout += 1
    #print("Simulation Number " + str(times+1) + " completed.")
    times += 1
    
print('Number of wins: ' + str(wins))
print("Max Steps Taken During a Win: " + str(max(win_dist)-1))
print("Average Steps Taken During a Win: " + str((sum(win_dist)/wins)-1))
print('Number of losses: ' + str(losses))
print("Max Steps Taken During a Loss: " + str(max(loss_dist)-1))
print("Average Steps Taken During a Loss: " + str((sum(loss_dist)/losses)-1))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))

Number of wins: 2408
Max Steps Taken During a Win: 81
Average Steps Taken During a Win: 14.56312292358804
Number of losses: 591
Max Steps Taken During a Loss: 80
Average Steps Taken During a Loss: 18.209813874788495
Number of timeouts: 1
Success Rate: 0.8026666666666666


In [7]:
def Agent6(g):
    agent_pos = random.randint(1,50)
    predator_6 = Predator(agent_pos)
    prey_6 = Prey(agent_pos)
    fail = 0
    steps = 0
    path = []
    path.append(agent_pos)

    belief_vector =  [(1/50)] * 50    #initializing belief-states
    #we know position of predator at first timestep
    for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
        if i == predator_6.pos-1:
            belief_vector[i] = 1
        else:
            belief_vector[i] = 0

    while(fail == 0 and steps <= 100):
        max_prob = max(belief_vector)
        #print(max_prob)
        #getting max probability value in belief vector
        list_of_positions_max_prob = []
        
        for itr in range(len(belief_vector)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector[itr] == max_prob):
                list_of_positions_max_prob.append(itr+1)

        #chosing node with highest prob of having pred closest to agent
        pat_len = 50
        best_positions = []
        for position in list_of_positions_max_prob:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_positions.append(position)
                else:
                    best_positions = []
                    best_positions.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))        
        possible_pred_pos = random.choice(best_positions) #randomly selecting one of these closest positions and surveying it
        #print(possible_pred_pos)
        
        #survey and belief update
        if possible_pred_pos == predator_6.pos: #if we survey node and find predator
            for i in range(len(belief_vector)): #making probability of predator being in all other nodes 0
                if i == predator_6.pos-1:
                    belief_vector[i] = 1
                else:
                    belief_vector[i] = 0
        else:
                pred_not = 1-max_prob
                belief_vector[possible_pred_pos-1] = 0
                for i in range(len(belief_vector)):
                        belief_vector[i] = belief_vector[i]/pred_not

        #print(str(belief_vector) + ' first update during timestep')
        #print(sum(belief_vector))

        """quasi_1 = sum(belief_vector)
        #print(quasi)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_1"""
        #print(belief_vector)        

        list_of_positions_max_chance = []

        for itr in range(len(belief_vector)): #making a list of positions which have the highest probability of having the predator
            if(belief_vector[itr] == max(belief_vector)):
                list_of_positions_max_chance.append(itr+1)

        #chosing node with highest prob of having predator closest to us
        pat_len = 50
        best_chance = []
        for position in list_of_positions_max_chance:
            if (len(BFS(agent_pos, position, g.graph)) <= pat_len):
                if(len(BFS(agent_pos, position, g.graph)) == pat_len):
                    best_chance.append(position)
                else:
                    best_chance = []
                    best_chance.append(position)
                    pat_len = len(BFS(agent_pos, position, g.graph))

        predicted_pred_pos = random.choice(best_chance) #randomly selecting one of these closest positions and surveying it
        #Agent 2 from here on
        neighbors = g.graph[agent_pos]
        #neighbors.append(agent_pos)
        # moves = list of tuples - each path has a tuple with distance to prey, a distance to predator, and 
        # the neighbor they would move to for that path
        amazing_moves = []
        path_to_prey = []
        path_to_pred = []
        curr_dist_to_pred = len(BFS(agent_pos, predicted_pred_pos, g.graph))
        curr_dist_to_prey = len(BFS(agent_pos, prey_6.pos, g.graph))
        all_possible_moves = [(curr_dist_to_prey, curr_dist_to_pred, agent_pos)]

        
        for neighbour in neighbors:
            path_to_prey = BFS(neighbour, prey_6.pos, g.graph)
            path_to_pred = BFS(neighbour, predicted_pred_pos, g.graph)

            #keeping all moves handy
            tup2 = (len(path_to_prey), len(path_to_pred), neighbour)
            all_possible_moves.append(tup2)
            
            #making list of moves which only satisfy 1st condition of Agent 1
            if(len(path_to_pred) > curr_dist_to_pred and len(path_to_prey) < curr_dist_to_prey):
                tup3 = (len(path_to_prey), len(path_to_pred), neighbour)
                amazing_moves.append(tup3)                


        if(len(amazing_moves) > 0):             
            moves_sorted = sorted(amazing_moves, key=lambda t: (t[0], -t[1]))
            move_choice=[]
            for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]): 
                    move_choice.append(move)
            neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
            agent_pos = neighbour_choice[2]


        else: #dip into bag of all possible moves

            if (curr_dist_to_pred <= 3): #if predator too close and enough steps in the bag then run away from predator
                moves_sorted = sorted(all_possible_moves, key=lambda t: (-t[1], t[0]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]

            else: #if step count exceeds this threshold, irrespective of predator distance make a dash to the prey
                moves_sorted = sorted(all_possible_moves, key=lambda t: (t[0], -t[1]))
                move_choice=[]
                for move in moves_sorted: #to check if there are multiple /best/ neighbours which have the same distance to prey and predator
                    if (moves_sorted[0][0] == move[0] and moves_sorted[0][1] == move[1]):
                        move_choice.append(move)
                neighbour_choice = random.choice(move_choice) #selecting from these multiple neighbours at random
                agent_pos = neighbour_choice[2]                

        path.append(agent_pos)

        steps += 1

        if (agent_pos == predator_6.pos):
            fail = 1 #if agent has already moved to location of prey
            break
        elif (agent_pos == prey_6.pos):
            break
        else:#treat as survey and update belief
            pred_not = 1-belief_vector[agent_pos-1] #incase we dont find, make probability at that position 0 and divide entire array by probability of predator not being in that position
            belief_vector[agent_pos-1] = 0
            for i in range(len(belief_vector)):
                    belief_vector[i] = belief_vector[i]/pred_not
            
            #print(str(sum(belief_vector)) + " Update 2 Timestep: " + str(steps))
        #print(str(belief_vector) + ' first update during timestep')
        #print(sum(belief_vector))
        """quasi_3 = sum(belief_vector)
        #print(quasi)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi_3 #to ensure that belief vector sums up to 1, leaving no margin for error"""

        #move prey
        prey_6.prey_move(g)

        if (prey_6.pos == agent_pos): #if prey luckily moves to location of agent
            break
        
        #move predator
        predator_6.pred_distracted_move(g, agent_pos)

        if(predator_6.pos == agent_pos): #if predator catches agent
            fail = 1
            break
        #belief update for next timestep
        temp_belief_vector = belief_vector.copy() #keeping copy for correct calculations
        for i in range(len(belief_vector)):
            mag_sum=0 #predator wont be at same node during next timestep
            i_neighbours = g.graph[i+1] 
            for bloke in i_neighbours:
                bloke_neighbours = g.graph[bloke] #finding neighbours of neighbours
                worthy_blokes=[]
                shortest_distance = 50
                for b_n in bloke_neighbours:
                    worthy_bloke_path = BFS(b_n, agent_pos, g.graph) #performing bfs from these 2nd degree neighbours
                    #saving shortest path(s)
                    if(len(worthy_bloke_path) <= shortest_distance):
                        if(len(worthy_bloke_path) == shortest_distance):
                            worthy_blokes.append(b_n)
                        else:
                            worthy_blokes = []
                            worthy_blokes.append(b_n) #keep list of neighbours with shortest path
                            shortest_distance = len(worthy_bloke_path) 
                mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(bloke_neighbours))) #moving from node bloke to node i in random manner
                mag_sum = mag_sum + (0.6*(temp_belief_vector[bloke-1])*worthy_blokes.count(i+1)/len(worthy_blokes)) #moving from node bloke to node i because it is on the shortest path to Agent
                """neighbour_path = BFS(bloke, agent_pos, g.graph)  #finding shortest path to agent position
                #print(neighbour_path)
                if len(neighbour_path) > 1:
                    mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/len(g.graph[bloke])))
                    if (neighbour_path[1] == i+1): #if the next position after start in the shortest path is the current position
                        mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.6))
                #else:
                    #mag_sum = mag_sum + ((temp_belief_vector[bloke-1])*(0.4/(len(g.graph[bloke])-1)))"""
            belief_vector[i] = mag_sum #assigning probability
        
        quasi = sum(belief_vector)
        for i in range(len(belief_vector)):
            belief_vector[i] /= quasi
        #print(sum(belief_vector))
    
    if(steps > 100):
            fail = 2     
    return fail, path

In [11]:
wins = 0
losses = 0
timeout = 0
times = 0
win_dist = []
loss_dist = []

while times < 3000:
    g = Graph()
    graph = g.addEdge()
    while graph:
        graph = g.addEdge()
    res, path = Agent6(g)
    
    if(res == 0):
        wins += 1
        win_dist.append(len(path))
    elif(res == 1):
        losses += 1
        loss_dist.append(len(path))
    else:
        timeout += 1
    #print("Simulation Number " + str(times+1) + " completed.")
    times += 1
    
print('Number of wins: ' + str(wins))
print("Max Steps Taken During a Win: " + str(max(win_dist)-1))
print("Average Steps Taken During a Win: " + str((sum(win_dist)/wins)-1))
print('Number of losses: ' + str(losses))
print("Max Steps Taken During a Loss: " + str(max(loss_dist)-1))
print("Average Steps Taken During a Loss: " + str((sum(loss_dist)/losses)-1))
print('Number of timeouts: ' + str(timeout))
print("Success Rate: " + str((wins/times)))

Number of wins: 2787
Max Steps Taken During a Win: 69
Average Steps Taken During a Win: 10.036239684248295
Number of losses: 213
Max Steps Taken During a Loss: 47
Average Steps Taken During a Loss: 12.953051643192488
Number of timeouts: 0
Success Rate: 0.929
